In [1]:
####  Resnet FOR cifar-10
import torch
import torch.nn as nn
import torchvision
import torch.optim as optimiz
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
### for image processing
transformation = torchvision.transforms.Compose([
            torchvision.transforms.Pad(4),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomCrop(32),
            torchvision.transforms.ToTensor()   
            ])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='../datasets/',
                                             train=True, 
                                             transform=transformation,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../datasets/',
                                            train=False, 
                                            transform=torchvision.transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=32,
                                          shuffle=False)

Files already downloaded and verified


In [ ]:
l_rate = 0.002
batch_size = 32
num_classes=10
def conv33(in_chan, out_chan, stride=1):
    return nn.Conv2d(in_chan, out_chan, kernel_size=3,stride=stride,
                    padding=1, bias=False)
#####   Each res block
class Residual(nn.Module):
    def __init__(self, in_chan, out_chan, stride=1, downsample=None):
        super(Residual, self).__init__()
        self.conv1 = conv33(in_chan, out_chan, stride)
        self.bn1 = nn.BatchNorm2d(out_chan)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv33(out_chan, out_chan)
        self.bn2 = nn.BatchNorm2d(out_chan)
        self.downsample = downsample
    
    def forward(self, x):
        res = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            res = self.downsample(x)
        out+=res
        out = self.relu(out)
        return out


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_chan = 16
        self.conv = conv33(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_chan, blocks, stride=1):
        downsample = None
        if(stride!=1) or (self.in_chan!=out_chan):
            downsample = nn.Sequential(
            conv33(self.in_chan, out_chan, stride=stride),
            nn.BatchNorm2d(out_chan))
        layers = []
        layers.append(block(self.in_chan, out_chan, stride, downsample))
        self.in_chan = out_chan
        for i in range(1, blocks):
            layers.append(block(out_chan, out_chan))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = ResNet(Residual, [2, 2, 2]).to(device)

loss = nn.CrossEntropyLoss()
optimizer = optimiz.Adam(model.parameters(), lr=l_rate)
def lr_update(optimizer, lr):
    for gr in optimizer.param_groups:
        gr['lr'] = lr

total_step = len(train_loader)
curr_lr = l_rate
num_epochs = 2
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss_val = loss(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss_val.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        lr_update(optimizer, curr_lr)



In [ ]:
model.eval()
with torch.no_grad():
    corr = 0
    tot = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        tot += labels.size(0)
        corr += (predicted == labels).sum().item()
    print('Accuracy of the model on the test images: {} %'.format(100 * corr / tot))
# Save the model checkpoint
torch.save(model.state_dict(), '../datasets/models/torch105.ckpt')